## Programación Entera
### Réplica de un Fondo Indice

Este cuaderno presenta un ejemplo de como elegir un conjunto de valores para 
replicar el IBEX35. La idea consiste en elegir para cada componente del índice
un represente, que puede ser el propio valor o un valor semejante.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from collections import defaultdict

### Datos
Utilizaremos una ventana de un año de los precios de cierre del IBEX
para construir una matriz de correlación que nos determinará la semejanza
de comportamiento entre valores

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
close_dict = {tk: df.close for tk, df in stock_data.items()}
close_df = pd.DataFrame(close_dict)

In [ ]:
close_year = close_df.loc['2019-01-02':'2019-12-31']
close_year = close_year.dropna(axis=1)

In [ ]:
returns = np.log(close_year).diff()
stock_corr = returns.corr()

In [ ]:
stock_corr

___

### Modelo de programación entera

In [ ]:
# numero de valores elegidos para la replica
n_fund = 20

In [ ]:
# numero de valores en el indice
n = stock_corr.shape[0]

**Variables de decisión**
- Xs Matriz nxn de binarias. Cada fila representa el valor del índice, y la columna indica con que valor se va a replicar dentro del fondo de replica
- Ys Vector de n binarias para indicar si un valor se ha seleccionado para el fondo

In [ ]:
x = cp.Variable(stock_corr.shape, boolean=True)
y = cp.Variable(stock_corr.shape[0], boolean=True)

In [ ]:
# Funcion objetivo
objective = cp.sum(cp.multiply(x, stock_corr))

In [ ]:
# la suma del vector y debe ser la cantidad de valores seleccionados para el fondo
constraints =[
    cp.sum(y) == n_fund,
]

In [ ]:
# Para cada fila solo debemos seleccionar un valor
for i in range(n):
    c_i = cp.sum(x[i,:]) == 1
    constraints.append(c_i)

In [ ]:
# Si un valor en el vector y no es seleccionado, su fila correspondiente
# en la matriz de Xs debe estar vacia 
for i in range(n):
    for j in range(n):
        c_ij = x[i,j] <= y[j]
        constraints.append(c_ij)

In [ ]:
prob = cp.Problem(cp.Maximize(objective), constraints)
res = prob.solve()
res

In [ ]:
## seleccionamos los valores que han sido elegidos por el modelo
selected = pd.Series(y.value, index=stock_corr.columns)
selected[selected == 1]

podemos ver que en cada fila solo hay un elemento seleccionado

In [ ]:
x.value[1,:]

In [ ]:
# recuperamos para cada fila en que posicion ha ocurrido el 1
occurr = np.argwhere(x.value == 1)
occurr

In [ ]:
# a partir de las ocurrencias reconstruimos
# que accion esta representado a cada componente del IBEX

matches = {tk: stock_corr.columns[occurr[i, 1]] 
           for i, tk in enumerate(stock_corr.columns)}

In [ ]:
matches

podemos reconstruir lo mismo buscando por columnas que elementos
representa cada acción.

In [ ]:
col_represent = np.argwhere(x.value.T == 1)
col_represent

In [ ]:
tickers = stock_corr.columns
group_represent = defaultdict(list)
for pair in represent:
    irep = tickers[pair[0]]
    istock = tickers[pair[1]]
    group_represent[irep].append(istock)
group_represent

___

### Ejercicio Propuesto

- Analizar el resultado del ejemplo anterior verificando diferentes numeros de acciones en el fondo de replica 
- Modificar el problema para que ningún valor pueda representar más de 3 activos a la vez